In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/RecFldGrn


# (Use Tokenizer)

In [2]:
sent = 'Her log of BP readings show downward trend.'

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
d = tokenizer(sent)
d['input_ids']

[101, 1430, 9366, 1104, 21062, 17813, 1437, 14833, 10209, 119, 102]

In [3]:
print([tokenizer.convert_ids_to_tokens(i) for i in d['input_ids']])

['[CLS]', 'Her', 'log', 'of', 'BP', 'readings', 'show', 'downward', 'trend', '.', '[SEP]']


In [4]:

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
sequence = sent
tokens = tokenizer.tokenize(sequence)
print(tokens)

['Her', 'log', 'of', 'BP', 'readings', 'show', 'downward', 'trend', '.']


# Read Raw Data

In [5]:
import os 
import pandas as pd

RecName = 'PNSectSent'
folder = 'data/Simulations/'
raw_file_path =  os.path.join(folder, RecName) + '.csv'
print(raw_file_path)


data/Simulations/PNSectSent.csv


In [6]:
# load all records
df = pd.read_csv(raw_file_path)
df

,PID,ECID,PNID,PNSectID,SentID,Sentence
0,P0,EC21,72374,149,1293,Later available sure scientist water out less.
1,P0,EC21,72374,145,1272,Wear relationship down surface section.
2,P0,EC21,72374,145,1271,Concern painting able than.
3,P0,EC21,72374,145,1270,Quite together just fund field.
4,P0,EC21,72374,145,1269,Owner PM building win cultural agent forward.
...,...,...,...,...,...,...
2582,P7,EC17,435,43,372,Man artist food area character.
2583,P7,EC17,435,43,373,Language very back strategy.
2584,P7,EC17,435,43,374,Feel budget tough edge read collection however...
2585,P7,EC17,435,39,310,Car true daughter guy strategy.


# Preprocess

## Adding Missing Records

In [7]:
from recfldgrn.datapoint import load_df_data_from_folder

prefix_cols = ['PID', 'ECID', 'PNID', 'PNSectID']
focal_cols = ['SentID']

Parent_RecName = 'PNSect'
rec_folder = 'data/ProcData/RecFolder/'
fullrec_folder =  os.path.join(rec_folder, Parent_RecName) 
df_parent = load_df_data_from_folder(fullrec_folder)
df_parent

,PID,ECID,PNID,PNSectID,SectName
0,P0,EC18,71800,190,Health Maintenance
1,P0,EC18,71800,188,Social History
2,P0,EC18,71800,178,Chief Complaint
3,P0,EC18,71800,179,History of Present Illness
4,P0,EC18,71800,180,Review of Systems
...,...,...,...,...,...
480,P7,EC17,435,44,Problem List/Past Medical History
481,P7,EC17,435,43,Assessment/Plan
482,P7,EC17,435,42,Physical Exam
483,P7,EC17,435,41,Review of Systems


In [8]:
import numpy as np

d = pd.merge(df_parent[prefix_cols], df, on = prefix_cols, how = 'left')
for col in focal_cols: 
    d.loc[d[col].isnull(), col] = np.ones(d[col].isnull().sum()).cumsum() + d[col].max()
    d[col] = d[col].astype(int)
    assert len(d[col]) == d[col].nunique()
    
df = d
df

,PID,ECID,PNID,PNSectID,SentID,Sentence
0,P0,EC18,71800,190,1547,Question religious small evidence.
1,P0,EC18,71800,190,1546,Prove subject responsibility.
2,P0,EC18,71800,190,1545,Card machine know fight experience option.
3,P0,EC18,71800,190,1544,Bank message several deal option be.
4,P0,EC18,71800,190,1543,Skin despite security set we.
...,...,...,...,...,...,...
2785,P7,EC17,435,41,321,More nothing scene leave chance tax.
2786,P7,EC17,435,40,312,Long follow account dream life.
2787,P7,EC17,435,40,313,Policy capital piece degree soon already.
2788,P7,EC17,435,40,314,Concern mind near some.


# Save To Disk by Bucket

In [9]:
from recfldgrn.datapoint import convert_PID_to_PIDgroup
from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder

In [10]:
rec_folder = 'data/ProcData/RecFolder'
fullrec_folder = os.path.join(rec_folder, RecName)
print(df.shape, fullrec_folder)

if not os.path.exists(fullrec_folder): os.makedirs(fullrec_folder)
write_df_to_folders(RecName, fullrec_folder, df)


(2790, 6) data/ProcData/RecFolder/PNSectSent


In [11]:
from recfldgrn.datapoint import PatientDP
print(rec_folder)
PID = 'P4'
Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
print(Pat)
dfx = Pat.get_df_rec(RecName)
dfx

data/ProcData/RecFolder


,PID,ECID,PNID,PNSectID,SentID,Sentence
0,P4,EC125,72530,437,2742,NaN
1,P4,EC24,72531,438,2743,NaN
2,P4,EC36,72532,439,2744,NaN
3,P4,EC80,72533,440,2745,NaN
4,P4,EC163,72534,441,2746,NaN
...,...,...,...,...,...,...
239,P4,EC121,72545,452,2757,NaN
240,P4,EC196,72546,453,2758,NaN
241,P4,EC203,72547,454,2759,NaN
242,P4,EC204,72548,455,2760,NaN


In [12]:
from recfldgrn.datapoint import load_df_data_from_folder
print(fullrec_folder)
df = load_df_data_from_folder(fullrec_folder)
df

data/ProcData/RecFolder/PNSectSent


,PID,ECID,PNID,PNSectID,SentID,Sentence
0,P0,EC18,71800,190,1547,Question religious small evidence.
1,P0,EC18,71800,190,1546,Prove subject responsibility.
2,P0,EC18,71800,190,1545,Card machine know fight experience option.
3,P0,EC18,71800,190,1544,Bank message several deal option be.
4,P0,EC18,71800,190,1543,Skin despite security set we.
...,...,...,...,...,...,...
2785,P7,EC17,435,41,321,More nothing scene leave chance tax.
2786,P7,EC17,435,40,312,Long follow account dream life.
2787,P7,EC17,435,40,313,Policy capital piece degree soon already.
2788,P7,EC17,435,40,314,Concern mind near some.


# Field & Grain

In [13]:
FieldGrainInfo = []

In [14]:
RecName = 'PNSectSent'
prefix_ids = ['PID', 'ECID', 'PNID', 'PNSectID']
focal_ids = ['SentID']
field_list = ['Sentence']

## Field `Sentence`

In [15]:
# (a) field and (b) get field-focused df
field = 'Sentence'
dfx = df[prefix_ids + focal_ids + [field]]
dfx

,PID,ECID,PNID,PNSectID,SentID,Sentence
0,P0,EC18,71800,190,1547,Question religious small evidence.
1,P0,EC18,71800,190,1546,Prove subject responsibility.
2,P0,EC18,71800,190,1545,Card machine know fight experience option.
3,P0,EC18,71800,190,1544,Bank message several deal option be.
4,P0,EC18,71800,190,1543,Skin despite security set we.
...,...,...,...,...,...,...
2785,P7,EC17,435,41,321,More nothing scene leave chance tax.
2786,P7,EC17,435,40,312,Long follow account dream life.
2787,P7,EC17,435,40,313,Policy capital piece degree soon already.
2788,P7,EC17,435,40,314,Concern mind near some.


### Grain `BasicDftGrn`

In [16]:
from recfldgrn.grains import Field2Grain_2_get_GrnStrList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnIdxList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnWgtList_Fn
from recfldgrn.utils import generate_grain_vocab_info

In [17]:
# (c) grain
grain = 'Tk@TknzLLMGrn'
field2grain = '-'.join([field, grain])
recfield2grain = RecName + '@' + field2grain
print(grain)
print(field2grain)
print(recfield2grain)

Tk@TknzLLMGrn
Sentence-Tk@TknzLLMGrn
PNSectSent@Sentence-Tk@TknzLLMGrn


In [18]:
from recfldgrn.grains.misc.dt_grn import load_DTgrn_Dictionary
from transformers import AutoTokenizer

if 'Nume' in grain:
    # print('Current grain is a numeric grain')
    grain_wgt_fn = Field2Grain_2_get_GrnWgtList_Fn[recfield2grain]
    series_wgt = dfx[field].apply(grain_wgt_fn)
    Vocab = {'v2idx': len(series_wgt.iloc[0]) * [1]}
    v2idx = Vocab['v2idx']

elif 'DT' in grain:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = load_DTgrn_Dictionary()
    v2idx = Vocab['v2idx']
    
elif 'Tknz' in grain:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    v2idx = tokenizer
    Vocab = {'v2idx': v2idx}

else:
    grain_str_fn = Field2Grain_2_get_GrnStrList_Fn[recfield2grain]
    series_str = dfx[field].apply(grain_str_fn)
    Vocab = generate_grain_vocab_info(series_str)
    v2idx = Vocab['v2idx']

print(v2idx)
grain_idx_fn = Field2Grain_2_get_GrnIdxList_Fn[recfield2grain]
series_idx = dfx[field].apply(lambda x: grain_idx_fn(x, v2idx))
series_idx


BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


0                [101, 3160, 3412, 2235, 3350, 1012, 102]
1                      [101, 6011, 3395, 5368, 1012, 102]
2       [101, 4003, 3698, 2113, 2954, 3325, 5724, 1012...
3       [101, 2924, 4471, 2195, 3066, 5724, 2022, 1012...
4          [101, 3096, 2750, 3036, 2275, 2057, 1012, 102]
                              ...                        
2785    [101, 2062, 2498, 3496, 2681, 3382, 4171, 1012...
2786       [101, 2146, 3582, 4070, 3959, 2166, 1012, 102]
2787    [101, 3343, 3007, 3538, 3014, 2574, 2525, 1012...
2788             [101, 5142, 2568, 2379, 2070, 1012, 102]
2789       [101, 2219, 4067, 2370, 2306, 2925, 1012, 102]
Name: Sentence, Length: 2790, dtype: object

In [19]:
d = {}
d['RecName'] = RecName

d['prefix_ids'] = prefix_ids
d['focal_ids'] = focal_ids 
d['field_list'] = field_list

d['field'] = field
d['grain'] = grain
d['field2grain'] = field2grain
d['recfield2grain'] = recfield2grain
d['Vocab'] = Vocab

FieldGrainInfo.append(d)

# Save Field Grain Info



In [20]:
df_FieldGrainInfo = pd.DataFrame(FieldGrainInfo)
df_FieldGrainInfo

,RecName,prefix_ids,focal_ids,field_list,field,grain,field2grain,recfield2grain,Vocab
0,PNSectSent,"[PID, ECID, PNID, PNSectID]",[SentID],[Sentence],Sentence,Tk@TknzLLMGrn,Sentence-Tk@TknzLLMGrn,PNSectSent@Sentence-Tk@TknzLLMGrn,{'v2idx': BertTokenizerFast(name_or_path='bert...


In [21]:
fldgrn_folder = 'data/ProcData/FldGrnInfo'

if not os.path.exists(fldgrn_folder): os.makedirs(fldgrn_folder)

fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')
df_FieldGrainInfo.to_pickle(fullfldgrn_file)